In [1]:
import os
import random
import time

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sympy.physics.units import length
from torchvision import transforms, models
from torchvision.models import vgg
from torchvision.transforms.functional import to_pil_image

from collections import defaultdict
from optuna import trial

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset, ConcatDataset
import optuna
import wandb
# Project utilities
import utils
import importlib
import train
importlib.reload(train)
importlib.reload(utils)
from train import train_model_with_hyperparams

VGG19 = 'VGG19'
ALEXNET = 'AlexNet'

# Set seed
SEED = utils.SEED
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Check if you're working locally or not
if not (os.path.exists(utils.CSV_PATH) and os.path.exists(utils.OPTIMIZED_DIR)):
    print(f"[!] You are NOT on the project's directory [!]\n"
          f"Please run the following command (in either CMD or anaconda prompt): \n"
          f"jupyter notebook --notebook-dir PROJECT_DIR\n"
          r"Where PROJECT_DIR is the project's directory in your computer e.g: C:\Users\amitr5\PycharmProjects\deep_van_gogh")

### Loading our data
We will load the optimized datasets from our custom dataset object


In [3]:
class NumPyDataset(Dataset):
    def __init__(self, file_path):
        data = np.load(file_path)
        self.images = data["images"]
        self.labels = data["labels"]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        idx = idx % len(self.images)
        x = torch.tensor(self.images[idx], dtype=torch.float32)
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y

dataset = NumPyDataset(os.path.join(utils.OPTIMIZED_DIR, 'dataset.npz'))

You can find the optimized dataset files <a href="https://drive.google.com/drive/folders/16vIyBwzvGgC-bJObJZT-RgZJmh3fj4Vt?usp=drive_link">HERE inside the data folder</a>. Note that you must have the data folder in the project directory<br/>
Loading the train and test datasets:

In [4]:
classes = pd.read_csv(utils.CSV_PATH)
train_rows = classes[classes['subset'] == 'train']

# train_indices, val_indices = train_test_split(train_rows.index.to_list(), test_size=0.2, random_state=utils.SEED, stratify=train_rows['is_van_gogh'])
# train_dataset = Subset(dataset, train_indices)
# val_dataset = Subset(dataset, val_indices)
train_dataset = Subset(dataset, train_rows.index.tolist())
test_dataset = Subset(dataset, classes[classes['subset'] == 'test'].index.tolist())

In [5]:
def get_opt_dataset(dataset_name):
    return NumPyDataset(os.path.join(utils.OPTIMIZED_DIR, f'{dataset_name}.npz'))

flip_dataset = get_opt_dataset('flip')
dropout_dataset = get_opt_dataset('dropout')
affine_dataset = get_opt_dataset('affine')
blur_dataset = get_opt_dataset('blur')
augmented_train_dataset = ConcatDataset([train_dataset, flip_dataset, dropout_dataset, affine_dataset, blur_dataset])

### Data Augmentation

For a detailed explanation about our data augmentation, please check augmentation_demo.ipynb

# Fine tuning VGG19

In [6]:
class FinedTunedModel(nn.Module):
    def __init__(self, base_model, architecture:str):
        super(FinedTunedModel, self).__init__()
        self._architecture = architecture  # Save the base model architecture
        base_children_list = list(base_model.children())
        self.features_extractor = nn.Sequential(*base_children_list[:-1]).to(device)
        for param in self.features_extractor.parameters():
            param.requires_grad = False

        # Modify the classifier to fit to our problem (2 classes)
        self.classifier = nn.Sequential(*base_children_list[-1])
        self.classifier[-1] = nn.Linear(4096, 2).to(device)  # Replaces the final layer of the base model's classifier with a new fully connected layer

    def forward(self, x):
        base_model_output = self.features_extractor(x)
        return self.classifier(torch.flatten(base_model_output, start_dim=1))
    @property
    def architecture(self):
        return self._architecture

In [7]:
# Load pre-trained models
vgg19 = models.vgg19(weights=models.VGG19_Weights.DEFAULT).to(device)
alexnet = models.alexnet(weights=models.AlexNet_Weights.DEFAULT).to(device)

In [8]:
def cross_validation(learning_rate,
                     weight_decay,
                     num_layers_finetune,
                     criterion,
                     epochs,
                     patience,
                     device,
                     architecture,
                     batch_size=128, trial=None, project='project'):
    k_folds = 4
    kfold = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

    # Create a label list for the augmented dataset
    labels = [train_rows['is_van_gogh']]
    for i in range(1, len(augmented_train_dataset.datasets)):
        ds = augmented_train_dataset.datasets[i]
        labels.append(ds.labels)
    labels = np.concatenate(labels)


    # Track performance for each model
    base_model = vgg19 if architecture == VGG19 else alexnet
    metrics_list = []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(np.zeros(len(labels)), labels)):
        wandb.init(project = project,
                       config={ "learning_rate": learning_rate,
                                "weight_decay": weight_decay,
                                "patience": patience,
                                "batch_size": batch_size,
                                "epochs": epochs,
                                "num_layers_finetune": num_layers_finetune,
                                "fold": fold + 1,
                                "trial": trial.number + 1,
                                "architecture": architecture,
                                "dataset": "Post_Impressionism",
                                }, name=f'{architecture}_trial_{trial.number + 1 if trial else -1}_fold_{fold+1}')

        train_subset = Subset(augmented_train_dataset, train_idx)
        val_subset = Subset(augmented_train_dataset, val_idx)

        # Create data loaders
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        model = FinedTunedModel(base_model, architecture).to(device)
        if num_layers_finetune:
            for param in model.features_extractor[-num_layers_finetune:].parameters():
                param.requires_grad = True

        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        # Train the model
        mean_metrics = train_model_with_hyperparams(model,
                                                train_loader,
                                                val_loader,
                                                optimizer,
                                                criterion,
                                                epochs=epochs,
                                                patience=patience,
                                                device=device,
                                                trial=trial,
                                                architecture=architecture, fold=fold + 1)
        metrics_list.append(mean_metrics)
        # Finish the Weights & Biases run
        wandb.finish()
    mean_dict = utils.mean_dict(metrics_list)
    return mean_dict


In [9]:
def objective(trial, architecture, config: dict) -> float:
    """
    Generic Optuna objective function.
    :param trial: Optuna trial object.
    :param model: The neural network model to train
    :param config: A dictionary with configurable values such as learning rate ranges, batch size ranges, etc.
    :return:  best_val_loss: The best validation loss achieved during training.
    """
    # Hyperparameter suggestions based on config
    learning_rate = trial.suggest_float("learning_rate",
                                        config.get("lr_min", 1e-5),
                                        config.get("lr_max", 1e-3),
                                        log=True)
    weight_decay = trial.suggest_float("weight_decay",
                                       config.get("wd_min", 1e-6),
                                       config.get("wd_max", 1e-4),
                                       log=True)
    # Including the option not to perform fine-tuning, or only a small num of layers within the feature extractor.
    num_layers_finetune = trial.suggest_int("num_layers_finetune", 0, 4)
    batch_min = config.get("batch_size_min", 32)
    batch_max =config.get("batch_size_max", 256)
    batch_size = trial.suggest_categorical("batch_size",
                                   [2**i for i in range(int(np.log2(batch_min)), int(np.log2(batch_max))+1)]
                                   )

    epochs = trial.suggest_int("epochs", config.get("epochs_min", 10), config.get("epochs_max",30))
    # patience = trial.suggest_int("patience", config.get("patience_min", 5), config.get("patience_max", 15))
    patience = 5

    # Define optimizer and loss function
    criterion = config.get("criterion", nn.CrossEntropyLoss()) # Classification.

    project = config.get("project", 'deep_van_gogh_default')
    # Train the model and get the best mean val_auc
    mean_dict = cross_validation(learning_rate, weight_decay, num_layers_finetune, criterion, epochs, patience, device, architecture, batch_size, trial, project=project)
    # Log the mean values

    wandb.init(project = project,
                       config={ "learning_rate": learning_rate,
                                "weight_decay": weight_decay,
                                "patience": patience,
                                "batch_size": batch_size,
                                "epochs": epochs,
                                "num_layers_finetune": num_layers_finetune,
                                "trial": trial.number + 1,
                                "architecture": architecture,
                                "dataset": "Post_Impressionism",
                                }, name=f"{architecture}_trial_{trial.number + 1 if trial else -1}")

    wandb.log(mean_dict)
    wandb.finish()
    # Return best validation loss as the objective to minimize
    return mean_dict['Validation AUC']



## Cross-Validation

In [ ]:
n_trials = 15
study = optuna.create_study(study_name=VGG19, direction='maximize')
study.optimize(lambda trial: objective(trial, VGG19, config={'project': 'VGG-19-CV_FINAL_Run'}), n_trials=n_trials)

In [10]:
def get_hyperparameters_from_best_model(architecture, trial_num, fold_num):
    hyperparameters = torch.load(f"models/{architecture}/{architecture}_best_model_trial_{trial_num}_fold_{fold_num}.pt",
                            weights_only=False)['hyperparameters']
    return hyperparameters

print('Best trial:')
# hyperparameters = get_hyperparameters_from_best_model(VGG, 2, 1)
# hyperparameters

Best trial:


In [97]:
train_indices, val_indices = train_test_split(train_rows.index.to_list(), test_size=0.2, random_state=utils.SEED, stratify=train_rows['is_van_gogh'])
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

In [11]:
def train_model_from_cv(architecture,
                        hyperparameters,
                        project_name = 'Final_Models_Run',
                        log = True,
                        save_model = True):
    base_model = alexnet if architecture == ALEXNET else vgg
    model = FinedTunedModel(base_model.to(device), architecture).to(device)
    weight_decay = hyperparameters['weight_decay']
    lr = hyperparameters['learning_rate']
    batch_size = hyperparameters['batch_size']
    epochs = hyperparameters['epochs']
    patience = hyperparameters['patience']
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()

    train_loader = DataLoader(augmented_train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    wandb.init(project=project_name,
               config={ "learning_rate": lr,
                        "weight_decay": weight_decay,
                        "patience": patience,
                        "batch_size": batch_size,
                        "epochs": epochs,
                        "architecture": architecture,
                        "dataset": "Post_Impressionism",
                        }, name=f"{architecture}-best")

    train_model_with_hyperparams(model, train_loader, val_loader, optimizer, criterion, epochs=epochs, patience=patience,device=device, trial=None, architecture=architecture, fold=-1, save_model=True, log=True)
    mean_values = train.validation(model, criterion, test_loader, device, is_test=True)
    wandb.log(mean_values)
    wandb.finish()
    return model



# train_model_from_cv(VGG19, hyperparameters,project_name = 'Final_Models_RUN')

# Fine tuning AlexNet

In [ ]:
n_trials = 15
study = optuna.create_study(study_name=ALEXNET, direction='maximize')
study.optimize(lambda trial: objective(trial, ALEXNET, config={'project':'AlexNet-CV_FINAL_Run'}), n_trials=n_trials)

## Training the model - ALEXNET

In [ ]:
print('Best trial:')
# hyperparameters = get_hyperparameters_from_best_model(VGG, 2, 1)
# hyperparameters

In [ ]:
# train_model_from_cv(ALEXNET, hyperparameters,project_name = 'Final_Models_RUN')

In [102]:
# Training the model - ALEXNET
# base_model = alexnet
# alexnet_model = FinedTunedModel(base_model.to(device), ALEXNET).to(device)
# weight_decay = hyperparameters['weight_decay']
# lr = hyperparameters['learning_rate']
# batch_size = hyperparameters['batch_size']
# epochs = hyperparameters['epochs']
# patience = hyperparameters['patience']
# optimizer = optim.Adam(alexnet_model.parameters(), lr=lr, weight_decay=weight_decay)
# criterion = nn.CrossEntropyLoss()
#
# train_loader = DataLoader(augmented_train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
#
# wandb.init(project='Final_Models_Run',
#                        config={ "learning_rate": lr,
#                                 "weight_decay": weight_decay,
#                                 "patience": patience,
#                                 "batch_size": batch_size,
#                                 "epochs": epochs,
#                                 "architecture": ALEXNET,
#                                 "dataset": "Post_Impressionism",
#                                 }, name=f"{ALEXNET}-best")
#
# train_model_with_hyperparams(alexnet_model, train_loader, val_loader, optimizer, criterion, epochs=epochs, patience=patience,device=device, trial=None, architecture=ALEXNET, fold=-1, save_model=True, log=True)
#
# best_values = train.validation(alexnet_model, criterion, test_loader, device, is_test=True)
# wandb.log(best_values)
# wandb.finish()


KeyError: 'batch_size'

analysing results

# Style transfer function